# 3D Droplet Oscillation

Results published: hopefully at some point!

It is part of the BoSSS-long-term validation test suite, which consists of 
several computationally expensive test-cases (runtime in the order of days),
which are performed on a regular basis in order to validate the 
physical correctness of BoSSS simulations.


### Preliminaries

This example can be found in the source code repository as as `Droplet3D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [2]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` namespace:

In [3]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;

Initialization of the Workflow management; there `ConfinedCylinder_ConvergenceStudy` is the project name which is used for all comutations (aka. sessions):

In [4]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database '\\fdygitrunner\ValidationTests\OscillatingDroplet3D'.


Overview on the available *Execution Queues* (aka. *Batch Processors*, aka. *Batch System*); these e.g. Linux HPC clusters on which compute jobs can be executed.

In [5]:
//wmg.Sessions[0].Export().WithSupersampling(2).Do()

In [6]:
wmg.Sessions

#0: OscillatingDroplet3D	J3456k3_arm1_mode2_aP0*	10/27/2021 18:14:50	d7700818...
#1: OscillatingDroplet3D	J432k3_arm1_mode2_aP0*	10/27/2021 18:14:39	e1789658...
#2: OscillatingDroplet3D	J3456k3_arm0_mode2_aP0*	10/27/2021 16:31:10	937cb519...
#3: OscillatingDroplet3D	J432k3_arm0_mode2_aP0*	10/27/2021 15:06:50	96f03adf...


In [7]:
wmg.Sessions[0].Timesteps

#0:  { Time-step: 0; Physical time: 0s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#1:  { Time-step: 1; Physical time: 5E-05s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#2:  { Time-step: 2; Physical time: 0.0001s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }


In [8]:
wmg.Sessions[1].Timesteps

#0:  { Time-step: 0; Physical time: 0s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#1:  { Time-step: 1; Physical time: 5E-05s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#2:  { Time-step: 2; Physical time: 0.0001s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#3:  { Time-step: 3; Physical time: 0.00015000000000000001s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#4:  { Time-step: 4; Physical time: 0.0002s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Me

In [9]:
wmg.Sessions[2].Timesteps

#0:  { Time-step: 0; Physical time: 0s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#1:  { Time-step: 1; Physical time: 5E-05s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#2:  { Time-step: 2; Physical time: 0.0001s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }


In [10]:
wmg.Sessions[3].Timesteps

#0:  { Time-step: 0; Physical time: 0s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#1:  { Time-step: 1; Physical time: 5E-05s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#2:  { Time-step: 2; Physical time: 0.0001s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#3:  { Time-step: 3; Physical time: 0.00015000000000000001s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Mean, Velocity0Z_Mean, VelocityX@Phi, VelocityY@Phi, VelocityZ@Phi; Name:  }
#4:  { Time-step: 4; Physical time: 0.0002s; Fields: Phi, PhiDG, VelocityX, VelocityY, VelocityZ, Pressure, Velocity0X_Mean, Velocity0Y_Me

In [13]:
wmg.Sessions[0].ID

d7700818-6cbf-481b-9353-2fce1b9dcf88